## Stock Prediction | LSTM
Google data

Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pi
import plotly.express as px
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta

In [2]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#### Load Stock Data
Load the stock data from the provided CSV file

In [6]:
stock_data = pd.read_csv('E:/Github Repositories/BharatIntern/Task/GOOG.csv')

In [7]:
stock_data.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,GOOG,2016-06-14 00:00:00+00:00,718.27,722.47,713.1200,716.48,1306065,718.27,722.47,713.1200,716.48,1306065,0.0,1.0
1,GOOG,2016-06-15 00:00:00+00:00,718.92,722.98,717.3100,719.00,1214517,718.92,722.98,717.3100,719.00,1214517,0.0,1.0
2,GOOG,2016-06-16 00:00:00+00:00,710.36,716.65,703.2600,714.91,1982471,710.36,716.65,703.2600,714.91,1982471,0.0,1.0
3,GOOG,2016-06-17 00:00:00+00:00,691.72,708.82,688.4515,708.65,3402357,691.72,708.82,688.4515,708.65,3402357,0.0,1.0
4,GOOG,2016-06-20 00:00:00+00:00,693.71,702.48,693.4100,698.77,2082538,693.71,702.48,693.4100,698.77,2082538,0.0,1.0


Data Ovreview

In [8]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   symbol       1258 non-null   object 
 1   date         1258 non-null   object 
 2   close        1258 non-null   float64
 3   high         1258 non-null   float64
 4   low          1258 non-null   float64
 5   open         1258 non-null   float64
 6   volume       1258 non-null   int64  
 7   adjClose     1258 non-null   float64
 8   adjHigh      1258 non-null   float64
 9   adjLow       1258 non-null   float64
 10  adjOpen      1258 non-null   float64
 11  adjVolume    1258 non-null   int64  
 12  divCash      1258 non-null   float64
 13  splitFactor  1258 non-null   float64
dtypes: float64(10), int64(2), object(2)
memory usage: 137.7+ KB


In [9]:
stock_data.describe().T

,count,mean,std,min,25%,50%,75%,max
close,1258.0,1.216317e+03,383.333358,668.260,9.608025e+02,1132.460,1.360595e+03,2521.60
high,1258.0,1.227431e+03,387.570872,672.300,9.687575e+02,1143.935,1.374345e+03,2526.99
low,1258.0,1.204176e+03,378.777094,663.284,9.521825e+02,1117.915,1.348557e+03,2498.29
open,1258.0,1.215261e+03,382.446995,671.000,9.590050e+02,1131.150,1.361075e+03,2524.92
volume,1258.0,1.601590e+06,696017.226844,346753.000,1.173522e+06,1412588.500,1.812156e+06,6207027.00
adjClose,1258.0,1.216317e+03,383.333358,668.260,9.608025e+02,1132.460,1.360595e+03,2521.60
adjHigh,1258.0,1.227431e+03,387.570873,672.300,9.687575e+02,1143.935,1.374345e+03,2526.99
adjLow,1258.0,1.204176e+03,378.777099,663.284,9.521825e+02,1117.915,1.348557e+03,2498.29
adjOpen,1258.0,1.215261e+03,382.446995,671.000,9.590050e+02,1131.150,1.361075e+03,2524.92
adjVolume,1258.0,1.601590e+06,696017.226844,346753.000,1.173522e+06,1412588.500,1.812156e+06,6207027.00


In [10]:
stock_data.isna().sum()

symbol         0
date           0
close          0
high           0
low            0
open           0
volume         0
adjClose       0
adjHigh        0
adjLow         0
adjOpen        0
adjVolume      0
divCash        0
splitFactor    0
dtype: int64

Data Visualization

In [11]:
fig = px.line(stock_data, x='date', y='volume', title='Daily Trading Volume')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Volume')
fig.update_layout(template='plotly_dark')
fig.show()

In [12]:
fig = px.line(stock_data, x='date', y='close', title='Closing Prices Over Time')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Closing Price')
fig.update_layout(template='plotly_dark')
fig.show()

In [13]:
fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['open'], mode='lines+markers', name='Open'))
fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['high'], mode='lines+markers', name='High'))
fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['low'], mode='lines+markers', name='Low'))
fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['close'], mode='lines+markers', name='Close'))

fig.update_layout(title='Stock Price Analysis',
                  xaxis_title='Date',
                  yaxis_title='Price')

fig.show()

In [14]:
daily_changes = stock_data['close'].diff()
fig = px.histogram(daily_changes, nbins=50, title='Histogram of Daily Price Changes')
fig.update_xaxes(title='Daily Price Change')
fig.update_yaxes(title='Frequency')
fig.update_layout(template='plotly_dark')
fig.show()

In [15]:
stock_data['20-day MA'] = stock_data['close'].rolling(window=20).mean()

fig = go.Figure(data=[go.Candlestick(
    x=stock_data['date'],
    open=stock_data['open'],
    high=stock_data['high'],
    low=stock_data['low'],
    close=stock_data['close'],
    name="Candlesticks",
    increasing_line_color='green',
    decreasing_line_color='red',
    line=dict(width=1),
    showlegend=False
)])

fig.add_trace(go.Scatter(x=stock_data['date'], y=stock_data['20-day MA'], mode='lines', name='20-day Moving Average', line=dict(color='rgba(255, 255, 0, 0.3)')))

fig.update_layout(
    title="Google Stock Price Candlestick Chart with Moving Average",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
)

fig.show()

In [16]:
stock_data = stock_data.drop('20-day MA', axis=1)

Data Preprocessing
1. Convert the 'date' column in the stock data to datetime format using pd.to_datetime.
2. Sort the stock data by date in ascending order using sort_values.
3. Create a new DataFrame 'stock' containing the selected columns (date, close, high, low, open, volume) for further analysis.

In [17]:
stock_data['date'] = pd.to_datetime(stock_data['date'])
stock_data = stock_data.sort_values('date')

In [18]:
stock = stock_data[['date', 'close', 'high', 'low', 'open', 'volume']]

Data Normalization
1. Initialize a Min-Max Scaler using MinMaxScaler().
2. Create a copy of the 'stock' DataFrame containing columns ('open', 'high', 'low', 'volume', 'close').
3. Use the scaler to fit and transform the data, performing Min-Max normalization.

In [19]:
scaler = MinMaxScaler()
normalized_data = stock[['open', 'high', 'low', 'volume', 'close']].copy()
normalized_data = scaler.fit_transform(normalized_data)

Data Splitting
1. Split the normalized data into training and testing sets using train_test_split. The testing set size is set to 20% of the data, and shuffle is set to False to maintain the chronological order.

2. Create a DataFrame 'train_df' containing the training data with columns (date, close, high, low, open, volume).

3. Create a DataFrame 'test_df' containing the testing data with columns (date, close, high, low, open, volume).

In [20]:
train_data, test_data = train_test_split(normalized_data, test_size=0.2, shuffle=False)

In [21]:
train_df = pd.DataFrame(train_data, columns=['open', 'high', 'low', 'volume', 'close'])
test_df = pd.DataFrame(test_data, columns=['open', 'high', 'low', 'volume', 'close'])

Sequence Generation
1. Define a function named generate_sequences that takes a DataFrame df and an optional parameter seq_length (default is 50).

2. Extract the relevant columns ('open', 'high', 'low', 'volume', 'close') from the DataFrame df and reset the index.

3. Initialize empty lists sequences and labels to store the sequences and labels for training.

4. Iterate through the data using a sliding window approach. For each index, append the next seq_length rows as a sequence and the corresponding last row as the label.

5. Convert the lists of sequences and labels into NumPy arrays.

6. Return the generated sequences and labels.

Then we generate sequences and labels for training data using generate_sequences function on 'train_df' DataFrame.

And next, we generate sequences and labels for testing data using generate_sequences function on 'test_df' DataFrame.

In [22]:
def generate_sequences(df, seq_length=50):
    X = df[['open', 'high', 'low', 'volume', 'close']].reset_index(drop=True)
    y = df[['open', 'high', 'low', 'volume', 'close']].reset_index(drop=True)

    sequences = []
    labels = []

    for index in range(len(X) - seq_length + 1):
        sequences.append(X.iloc[index : index + seq_length].values)
        labels.append(y.iloc[index + seq_length - 1].values)

    sequences = np.array(sequences)
    labels = np.array(labels)

    return sequences, labels

In [23]:
train_sequences, train_labels = generate_sequences(train_df)
test_sequences, test_labels = generate_sequences(test_df)

Model Architecture
1. Create a Sequential model.

2. Add the first LSTM layer with 50 units and return sequences. The input shape is set to (50, 5), where 50 is the sequence length and 5 is the number of features ('open', 'high', 'low', 'volume', 'close').

LSTM Layer 1:
Units: 50
Input Shape: (50, 5)
Return Sequences: True
3. Apply dropout regularization with a rate of 0.2 to mitigate overfitting.

Dropout Layer 1:
Rate: 0.2
4. Add the second LSTM layer with 50 units and return sequences.

LSTM Layer 2:
Units: 50
Return Sequences: True
5. Apply dropout regularization with a rate of 0.2.

Dropout Layer 2:
Rate: 0.2
6. Add the third LSTM layer with 50 units.

LSTM Layer 3:
Units: 50
7. Apply dropout regularization with a rate of 0.2.

Dropout Layer 3:
Rate: 0.2
8. Add a fully connected Dense layer with 5 units as the output layer.

Dense Layer:
Units: 5 (output)
This model architecture comprises multiple LSTM layers with dropout regularization to prevent overfitting.

In [24]:
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(50, 5)),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=5)
])

Compile and Summary
1. Compile the model for training using the mean squared error as the loss function and the Adam optimizer. Additionally, track the mean absolute error as a metric.

Loss Function: Mean Squared Error
Optimizer: Adam
Metrics: Mean Absolute Error
2. Display a summary of the model's architecture and parameters.

In [25]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 50)            11200     
                                                                 
 dropout (Dropout)           (None, 50, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 50, 50)            20200     
                                                                 
 dropout_1 (Dropout)         (None, 50, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 5)                 2

Model Training
1. Set the number of training epochs to 200 and the batch size to 32 for training the model.

2. Train the model using the training sequences and labels. During training, the model will run for the specified number of epochs, updating its weights to minimize the loss.

Number of Epochs: 200
Batch Size: 32
Training Data: train_sequences and train_labels
Validation Data: test_sequences and test_labels
Verbose: Display training progress information.

In [26]:
epochs = 200
batch_size = 32

history = model.fit(
    train_sequences,
    train_labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(test_sequences, test_labels),
    verbose=1
)

Epoch 1/200
30/30 [==============================] - 13s 146ms/step - loss: 0.0142 - mean_absolute_error: 0.0854 - val_loss: 0.0290 - val_mean_absolute_error: 0.1414
Epoch 2/200
30/30 [==============================] - 2s 79ms/step - loss: 0.0055 - mean_absolute_error: 0.0499 - val_loss: 0.0236 - val_mean_absolute_error: 0.1277
Epoch 3/200
30/30 [==============================] - 2s 79ms/step - loss: 0.0047 - mean_absolute_error: 0.0444 - val_loss: 0.0139 - val_mean_absolute_error: 0.0922
Epoch 4/200
30/30 [==============================] - 2s 82ms/step - loss: 0.0046 - mean_absolute_error: 0.0429 - val_loss: 0.0130 - val_mean_absolute_error: 0.0937
Epoch 5/200
30/30 [==============================] - 3s 85ms/step - loss: 0.0042 - mean_absolute_error: 0.0403 - val_loss: 0.0099 - val_mean_absolute_error: 0.0805
Epoch 6/200
30/30 [==============================] - 2s 79ms/step - loss: 0.0041 - mean_absolute_error: 0.0404 - val_loss: 0.0110 - val_mean_absolute_error: 0.0865
Epoch 7/200
30

In [27]:
train_predictions = model.predict(train_sequences)
test_predictions = model.predict(test_sequences)

7/7 [==============================] - 0s 28ms/step


Training Data Predictions

In [28]:
fig = make_subplots(rows=1, cols=1, subplot_titles=('Close Predictions'))

train_close_pred = train_predictions[:, 0]
train_close_actual = train_labels[:, 0]

fig.add_trace(go.Scatter(x=np.arange(len(train_close_actual)), y=train_close_actual, mode='lines', name='Actual', opacity=0.9))
fig.add_trace(go.Scatter(x=np.arange(len(train_close_pred)), y=train_close_pred, mode='lines', name='Predicted', opacity=0.6))

fig.update_layout(title='Close Predictions on Train Data', template='plotly_dark')
fig.show()

Next 10 Days Predictions
1. Initialize an empty list latest_prediction to store the model's predictions.

2. Extract the last sequence of the test data using test_sequences[:-1].

3. Loop 10 times to predict the next values. In each iteration, predict the next sequence using the model and append the prediction to latest_prediction.

In [29]:
latest_prediction = []
last_seq = test_sequences[:-1]

for _ in range(10):
    prediction = model.predict(last_seq)
    latest_prediction.append(prediction)

7/7 [==============================] - 0s 31ms/step


In [30]:
pi.templates.default = "plotly_dark"

predicted_data_next = np.array(latest_prediction).reshape(-1, 5)
last_date = stock['date'].max()
next_10_days = [last_date + timedelta(days=i) for i in range(1, 11)]

for i, feature_name in enumerate(['open', 'high', 'low', 'volume', 'close']):
    if feature_name in ['volume', 'close']:
        fig = go.Figure()

        fig.add_trace(go.Scatter(x=next_10_days, y=predicted_data_next[:, i],
                                 mode='lines+markers', name=f'Predicted {feature_name.capitalize()} Prices'))

        fig.update_layout(title=f'Predicted {feature_name.capitalize()} Prices for the Next 10 Days',
                          xaxis_title='Date', yaxis_title=f'{feature_name.capitalize()} Price')

        fig.show()